**Author:
Bibek Prasad Gupta**

In [31]:
import pandas as pd
import numpy as np
import math
import io
from tqdm import tqdm
from datetime import datetime

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [32]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [33]:
os.chdir('/content/drive/MyDrive/SparkIntern')

**Read the bitcoin dataset**

In [34]:
# data = pd.read_csv("bitstampUSD.csv", nrows=240000)
data = pd.read_csv("prices.csv")

In [35]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-06-14,17.410000,17.410000,17.410000,17.410,17.410,1.0
1,2016-06-15,17.490000,17.490000,17.489000,17.489,17.489,1.0
2,2016-06-16,17.700001,17.700001,17.254999,17.594,17.594,10.0
3,2016-06-17,17.420000,17.455000,17.400000,17.400,17.400,2.0
4,2016-06-20,17.445000,17.503000,17.445000,17.503,17.503,3.0


**Checking the columns which have null values and dropping the null rows**

In [36]:
# data[data.columns.values] = data[data.columns.values].ffill()
columns_to_check_for_null = ['Close']
data.shape
data.dropna(subset=columns_to_check_for_null, inplace=True)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-06-14,17.410000,17.410000,17.410000,17.410,17.410,1.0
1,2016-06-15,17.490000,17.490000,17.489000,17.489,17.489,1.0
2,2016-06-16,17.700001,17.700001,17.254999,17.594,17.594,10.0
3,2016-06-17,17.420000,17.455000,17.400000,17.400,17.400,2.0
4,2016-06-20,17.445000,17.503000,17.445000,17.503,17.503,3.0


In [37]:
data.shape

(1245, 7)

**COnvert to datetime datatype**

In [38]:
data["date"] = pd.to_datetime(data["Date"],format='%Y-%m-%d', errors= "coerce").dt.tz_localize(None)
data.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,date
0,2016-06-14,17.41,17.41,17.410,17.410,17.410,1.0,2016-06-14
1,2016-06-15,17.49,17.49,17.489,17.489,17.489,1.0,2016-06-15


In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1245 entries, 0 to 1255
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1245 non-null   object        
 1   Open       1245 non-null   float64       
 2   High       1245 non-null   float64       
 3   Low        1245 non-null   float64       
 4   Close      1245 non-null   float64       
 5   Adj Close  1245 non-null   float64       
 6   Volume     1245 non-null   float64       
 7   date       1245 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 87.5+ KB


**Create index for grouper and calculate close price by one hour frequency**

In [40]:
data['time'] = data['date']
data.index = data['time']

crypto_usd_grouped = data.groupby(pd.Grouper(key="time", freq="1D")).agg(
    closed_price_by_day = ('Close', np.mean)
)

In [41]:
crypto_usd_grouped.head()

,closed_price_by_day
time,
2016-06-14,17.410
2016-06-15,17.489
2016-06-16,17.594
2016-06-17,17.400
2016-06-18,NaN


**Method to check the upward and down trend**

In [42]:
def movement_classifier(x):
    try:
        if x >= 0 :
            return 1
        else:
            return 0
    except:
        print(x)

**Calculate the price trend**

In [43]:
# Drop null rows
crypto_usd_grouped.dropna(subset=['closed_price_by_day'], inplace=True)
# Calculate the difference between previous row
crypto_usd_grouped["price_diff"] = crypto_usd_grouped["closed_price_by_day"].diff()
# Calculate price movement
crypto_usd_grouped["movement"] = crypto_usd_grouped["price_diff"].progress_apply(lambda x:movement_classifier(x))
crypto_usd_grouped.head()

100%|██████████| 1245/1245 [00:00<00:00, 302771.99it/s]


,closed_price_by_day,price_diff,movement
time,,,
2016-06-14,17.410,NaN,0
2016-06-15,17.489,0.079,1
2016-06-16,17.594,0.105,1
2016-06-17,17.400,-0.194,0
2016-06-20,17.503,0.103,1


In [44]:
crypto_usd_grouped.shape

(1245, 3)

**Save data for future use**

In [45]:
os.chdir("/content/drive/MyDrive/SparkIntern")
crypto_usd_grouped.to_csv("price_processed_data.csv")

**Merge sentiment and bitcoin prices data**

In [46]:
# Run sentiment_preprocessing notebook to generate "twitter_sentiments_hourly_processed_data.csv" file
data_sentiments = pd.read_csv("sentiments_processed_data.csv")
data_sentiments["time"] = pd.to_datetime(data_sentiments["time"], errors="coerce")
data_bitcoin = pd.read_csv("price_processed_data.csv")
data_bitcoin["time"] = pd.to_datetime(data_bitcoin["time"], errors="coerce")
merged_data = pd.merge(data_bitcoin, data_sentiments, on='time')

**Save data for future use**

In [47]:
# Final features to train the models
merged_data.to_csv("merged_price_and_sentiments_data.csv")
merged_data.columns

Index(['time', 'closed_price_by_day', 'price_diff', 'movement',
       'Daily_Weight_mean_by_day', 'Daily_Weight_count_by_day',
       'blob_sent_mean_by_day', 'subjectivity_mean_by_day',
       'vader_sent_mean_by_day', 'news_diff', 'blob_sent_mean_by_day_diff',
       'vader_sent_mean_by_day_diff', 'news_movement', 'blob_sent_movement',
       'vader_sent_movement'],
      dtype='object')

In [48]:
merged_data.head()

,time,closed_price_by_day,price_diff,movement,Daily_Weight_mean_by_day,Daily_Weight_count_by_day,blob_sent_mean_by_day,subjectivity_mean_by_day,vader_sent_mean_by_day,news_diff,blob_sent_mean_by_day_diff,vader_sent_mean_by_day_diff,news_movement,blob_sent_movement,vader_sent_movement
0,2016-06-14,17.410,NaN,0,0.0,212,0.079657,0.408165,-0.048051,-21.0,0.009086,-0.046158,0,1,0
1,2016-06-15,17.489,0.079,1,0.0,173,0.089268,0.477367,0.012755,-39.0,0.009612,0.060806,0,1,1
2,2016-06-16,17.594,0.105,1,0.0,201,0.091617,0.419953,-0.101170,28.0,0.002349,-0.113926,1,1,0
3,2016-06-17,17.400,-0.194,0,0.0,193,0.065786,0.426227,0.014444,-8.0,-0.025831,0.115614,0,0,1
4,2016-06-20,17.503,0.103,1,0.0,197,0.069862,0.430430,-0.042131,-35.0,0.001532,-0.049915,0,1,0
